# **PROCESAMIENTO DE DATOS**

In [ ]:
archivo_modelo = "anuncios_resultado_modelo.json"
archivo_humano_1 = "anuncios_evaluados_humano_1.json"
archivo_humano_2 = "anuncios_evaluados_humano_2.json"
archivo_humano_3 = "anuncios_evaluados_humano_3.json"
archivo_humano = "anuncios_evaluados_humano_final.json"

In [ ]:
def eliminar_duplicados_por_id(datos):
    """Elimina entradas duplicadas por 'id' en una lista de objetos JSON."""
    ids_vistos = set()
    datos_limpios = []

    for item in datos:
        id_ = item.get("id")
        if id_ not in ids_vistos:
            ids_vistos.add(id_)
            datos_limpios.append(item)

    return datos_limpios

import json

# Cargar archivos JSON
with open(archivo_modelo, encoding="utf-8") as f:
    datos_modelo_raw = json.load(f)

with open(archivo_humano_1, encoding="utf-8") as f:
    datos_humano_raw_1 = json.load(f)
with open(archivo_humano_2, encoding="utf-8") as f:
    datos_humano_raw_2 = json.load(f)
with open(archivo_humano_3, encoding="utf-8") as f:
    datos_humano_raw_3 = json.load(f)


datos_modelo = eliminar_duplicados_por_id(datos_modelo_raw)
datos_humano_1 = eliminar_duplicados_por_id(datos_humano_raw_1)
datos_humano_2 = eliminar_duplicados_por_id(datos_humano_raw_2)
datos_humano_3 = eliminar_duplicados_por_id(datos_humano_raw_3)
print(f"Original modelo: {len(datos_modelo_raw)}, Sin duplicados: {len(datos_modelo)}")
print(f"Original evaluador 1: {len(datos_humano_raw_1)}, Sin duplicados: {len(datos_humano_1)}")
print(f"Original evaluador 2: {len(datos_humano_raw_2)}, Sin duplicados: {len(datos_humano_2)}")
print(f"Original evaluador 3: {len(datos_humano_raw_3)}, Sin duplicados: {len(datos_humano_3)}")


Original modelo: 2382, Sin duplicados: 2187
Original evaluador 1: 2382, Sin duplicados: 2187
Original evaluador 2: 2382, Sin duplicados: 2187
Original evaluador 3: 2382, Sin duplicados: 2187


In [ ]:
import json
from collections import Counter

# Función de conteo de etiquetas de agresividad
def contar_agresividad_humano(datos):
    conteo_analizado = Counter()
    conteo_imagen = Counter()

    for item in datos:
        analisis = item.get("analisis_modelos", {})

        # --- Texto analizado ---
        if "texto_analizado" in analisis:
            resultados = analisis["texto_analizado"].get("resultados", {})
            label = resultados.get("agresividad")
            if label is not None:
                conteo_analizado[label] += 1

        # --- Texto imagen ---
        if "texto_imagen" in analisis:
            resultados = analisis["texto_imagen"].get("resultados", {})
            label = resultados.get("agresividad")
            if label is not None:
                conteo_imagen[label] += 1

    return conteo_analizado, conteo_imagen

# Ejecutar conteo
humano_analizado_1, humano_imagen_1 = contar_agresividad_humano(datos_humano_1)
humano_analizado_2, humano_imagen_2 = contar_agresividad_humano(datos_humano_2)
humano_analizado_3, humano_imagen_3 = contar_agresividad_humano(datos_humano_3)

# Mostrar resultados
print("Total de etiquetas de AGRESIVIDAD\n")
print("\nHUMANO 1")
print("  Texto analizado:", dict(humano_analizado_1))
print("  Texto imagen:   ", dict(humano_imagen_1))

print("\nHUMANO 2")
print("  Texto analizado:", dict(humano_analizado_2))
print("  Texto imagen:   ", dict(humano_imagen_2))

print("\nHUMANO 3")
print("  Texto analizado:", dict(humano_analizado_3))
print("  Texto imagen:   ", dict(humano_imagen_3))

Total de etiquetas de AGRESIVIDAD


HUMANO 1
  Texto analizado: {0: 2133, 1: 43, 2: 11}
  Texto imagen:    {0: 521, 1: 30, 2: 5}

HUMANO 2
  Texto analizado: {0: 2080, 1: 96, 2: 11}
  Texto imagen:    {0: 453, 1: 98, 2: 5}

HUMANO 3
  Texto analizado: {0: 2047, 1: 38, 2: 102}
  Texto imagen:    {0: 502, 2: 39, 1: 15}


In [ ]:
from collections import defaultdict, Counter
from scipy.stats import entropy
import numpy as np
import math

def calcular_entropia_por_etiqueta(*listas_anotadores):
    valores_por_campo = {
        'texto_analizado': defaultdict(list),
        'texto_imagen': defaultdict(list),
    }

    for lista in listas_anotadores:
        for item in lista:
            for tipo_texto in ['texto_analizado', 'texto_imagen']:
                # Validar existencia de tipo de texto
                if tipo_texto not in item.get('analisis_modelos', {}):
                    continue
                resultados = item['analisis_modelos'][tipo_texto].get('resultados', {})
                for campo in ['formalidad', 'agresividad', 'edad', 'genero']:
                    valor = resultados.get(campo)
                    if valor is not None:
                        valores_por_campo[tipo_texto][campo].append(valor)

    def calcular_entropia(valores):
        conteo = Counter(valores)
        total = sum(conteo.values())
        probs = np.array([v / total for v in conteo.values()])
        return entropy(probs, base=2) if total > 0 else 0.0

    entropias = {}
    for tipo_texto, campos in valores_por_campo.items():
        entropias[tipo_texto] = {}
        for campo, valores in campos.items():
            entropias[tipo_texto][campo] = calcular_entropia(valores)

    return entropias

entropias = calcular_entropia_por_etiqueta(datos_humano_1, datos_humano_2, datos_humano_3)

for tipo, campos in entropias.items():
    print(f"\n▶ {tipo.upper()}")
    for campo, valor in campos.items():
        n_clases = 4 if campo == "edad" else 3
        H_max = math.log2(n_clases)
        H_normalizada = valor / H_max
        print(f"  - {campo}: {valor:.4f}")
        print(f"Entropía normalizada: {H_normalizada:.4f}")


▶ TEXTO_ANALIZADO
  - formalidad: 1.0210
Entropía normalizada: 0.6442
  - agresividad: 0.3135
Entropía normalizada: 0.1978
  - edad: 0.9508
Entropía normalizada: 0.4754
  - genero: 0.6161
Entropía normalizada: 0.3887

▶ TEXTO_IMAGEN
  - formalidad: 1.2261
Entropía normalizada: 0.7736
  - agresividad: 0.6095
Entropía normalizada: 0.3845
  - edad: 1.3082
Entropía normalizada: 0.6541
  - genero: 0.7399
Entropía normalizada: 0.4669


Generar el json a comparar:

In [ ]:
# Asignar un valor ponderado
import json
from collections import Counter

pesos = [1, 2, 2]

def ponderar_numerico(valores):
    return round(sum(p * v for p, v in zip(pesos, valores)) / sum(pesos))

def ponderar_categorico(valores):
    contador = Counter()
    for peso, valor in zip(pesos, valores):
        contador[valor] += peso
    return contador.most_common(1)[0][0]

resultados_finales = []
for a1, a2, a3 in zip(datos_humano_raw_1, datos_humano_raw_2, datos_humano_raw_3):
    resultado = {
        "id": a1["id"],
        "imagen_local": a1["imagen_local"],
        "fecha": a1["fecha"],
        "comercio": a1["comercio"],
        "analisis_modelos": {}
    }

    for campo in ["texto_analizado", "texto_imagen"]:
        if campo in a1["analisis_modelos"]:
            r1 = a1["analisis_modelos"][campo]["resultados"]
            r2 = a2["analisis_modelos"][campo]["resultados"]
            r3 = a3["analisis_modelos"][campo]["resultados"]

            resultados_ponderados = {
                "formalidad": ponderar_numerico([r1["formalidad"], r2["formalidad"], r3["formalidad"]]),
                "agresividad": ponderar_numerico([r1["agresividad"], r2["agresividad"], r3["agresividad"]]),
                "edad": ponderar_categorico([r1["edad"], r2["edad"], r3["edad"]]),
                "genero": ponderar_categorico([r1["genero"], r2["genero"], r3["genero"]])
            }

            resultado["analisis_modelos"][campo] = {
                "descripcion": a1["analisis_modelos"][campo]["descripcion"],
                "resultados": resultados_ponderados
            }

    resultados_finales.append(resultado)

with open(archivo_humano, "w", encoding="utf-8") as f:
    json.dump(resultados_finales, f, ensure_ascii=False, indent=2)


In [ ]:
import json

# Cargar archivos JSON
with open(archivo_modelo, encoding="utf-8") as f:
    datos_modelo_raw = json.load(f)

with open(archivo_humano, encoding="utf-8") as f:
    datos_humano_raw = json.load(f)

datos_modelo = eliminar_duplicados_por_id(datos_modelo_raw)
datos_humano = eliminar_duplicados_por_id(datos_humano_raw)

# Obtener sets de IDs
ids_modelo = {item["id"] for item in datos_modelo}
ids_humano = {item["id"] for item in datos_humano}

# Comparar
solo_en_modelo = ids_modelo - ids_humano
solo_en_humano = ids_humano - ids_modelo

# Mostrar resultados
print(f"🔍 IDs en modelo pero no en humano ({len(solo_en_modelo)}): {sorted(solo_en_modelo)}")
print(f"🔍 IDs en humano pero no en modelo ({len(solo_en_humano)}): {sorted(solo_en_humano)}")


🔍 IDs en modelo pero no en humano (0): []
🔍 IDs en humano pero no en modelo (0): []


# **COMPARACIONES**

## **FORMALIDAD**

In [ ]:
import json
from collections import Counter

def contar_formalidad(datos, fuente):
    conteo_analizado = Counter()
    conteo_imagen = Counter()

    for item in datos:
        analisis = item.get("analisis_modelos", {})

        # --- Texto analizado ---
        if "texto_analizado" in analisis:
            resultados = analisis["texto_analizado"].get("resultados", {})
            if fuente == "modelo":
                label = resultados.get("formalidad", {}).get("label")
            else:
                label = resultados.get("formalidad")
            if label is not None:
                conteo_analizado[label] += 1

        # --- Texto imagen ---
        if "texto_imagen" in analisis:
            resultados = analisis["texto_imagen"].get("resultados", {})
            if fuente == "modelo":
                label = resultados.get("formalidad", {}).get("label")
            else:
                label = resultados.get("formalidad")
            if label is not None:
                conteo_imagen[label] += 1

    return conteo_analizado, conteo_imagen

# Ejecutar conteo
modelo_analizado, modelo_imagen = contar_formalidad(datos_modelo, "modelo")
humano_analizado, humano_imagen = contar_formalidad(datos_humano, "humano")

# Mostrar resultados
print("Total de etiquetas de FORMALIDAD\n")
print("MODELO")
print("  Texto analizado:", dict(modelo_analizado))
print("  Texto imagen:   ", dict(modelo_imagen))
print("\nHUMANO")
print("  Texto analizado:", dict(humano_analizado))
print("  Texto imagen:   ", dict(humano_imagen))


Total de etiquetas de FORMALIDAD

MODELO
  Texto analizado: {0: 1616, 2: 565, 1: 6}
  Texto imagen:    {0: 405, 2: 151}

HUMANO
  Texto analizado: {1: 1707, 0: 313, 2: 167}
  Texto imagen:    {1: 382, 0: 107, 2: 67}


**Coincidencias modelo vs humano**

In [ ]:
import json
from collections import Counter

# Indexar por ID
modelo_dict = {item["id"]: item for item in datos_modelo}
humano_dict = {item["id"]: item for item in datos_humano}

# Comparar formalidad
def comparar_formalidad_por_id(modelo_dict, humano_dict):
    coinciden = {"texto_analizado": Counter(), "texto_imagen": Counter()}
    no_coinciden = {"texto_analizado": Counter(), "texto_imagen": Counter()}

    ids_comunes = set(modelo_dict.keys()) & set(humano_dict.keys())

    for id_ in set(modelo_dict.keys()) & set(humano_dict.keys()):
        m = modelo_dict[id_]
        h = humano_dict[id_]

        for campo in ["texto_analizado", "texto_imagen"]:
            m_res = m.get("analisis_modelos", {}).get(campo, {}).get("resultados", {})
            h_res = h.get("analisis_modelos", {}).get(campo, {}).get("resultados", {})

            label_modelo = m_res.get("formalidad", {}).get("label")
            label_humano = h_res.get("formalidad")

            if label_modelo is not None and label_humano is not None:
                if label_modelo == label_humano:
                    coinciden[campo][label_humano] += 1
                else:
                    no_coinciden[campo][label_humano] += 1

    return coinciden, no_coinciden

# Ejecutar
coinciden, no_coinciden = comparar_formalidad_por_id(modelo_dict, humano_dict)

# Mostrar resultados
print("Coincidencias:")
for campo, conteo in coinciden.items():
    print(f"  {campo}: {dict(conteo)}")

print("\nNo coincidencias:")
for campo, conteo in no_coinciden.items():
    print(f"  {campo}: {dict(conteo)}")


Coincidencias:
  texto_analizado: {0: 239, 2: 109, 1: 6}
  texto_imagen: {0: 51, 2: 33}

No coincidencias:
  texto_analizado: {1: 1701, 2: 58, 0: 74}
  texto_imagen: {1: 382, 0: 56, 2: 34}


## **AGRESIVIDAD**

In [ ]:
import json
from collections import Counter

# Función de conteo de etiquetas de agresividad
def contar_agresividad(datos, fuente):
    conteo_analizado = Counter()
    conteo_imagen = Counter()

    for item in datos:
        analisis = item.get("analisis_modelos", {})

        # --- Texto analizado ---
        if "texto_analizado" in analisis:
            resultados = analisis["texto_analizado"].get("resultados", {})
            if fuente == "modelo":
                label = resultados.get("agresividad", {}).get("label")
            else:
                label = resultados.get("agresividad")
            if label is not None:
                conteo_analizado[label] += 1

        # --- Texto imagen ---
        if "texto_imagen" in analisis:
            resultados = analisis["texto_imagen"].get("resultados", {})
            if fuente == "modelo":
                label = resultados.get("agresividad", {}).get("label")
            else:
                label = resultados.get("agresividad")
            if label is not None:
                conteo_imagen[label] += 1

    return conteo_analizado, conteo_imagen

# Ejecutar conteo
modelo_analizado, modelo_imagen = contar_agresividad(datos_modelo, "modelo")
humano_analizado, humano_imagen = contar_agresividad(datos_humano, "humano")

# Mostrar resultados
print("Total de etiquetas de AGRESIVIDAD\n")
print("MODELO")
print("  Texto analizado:", dict(modelo_analizado))
print("  Texto imagen:   ", dict(modelo_imagen))
print("\nHUMANO")
print("  Texto analizado:", dict(humano_analizado))
print("  Texto imagen:   ", dict(humano_imagen))


Total de etiquetas de AGRESIVIDAD

MODELO
  Texto analizado: {0: 2126, 1: 58, 2: 3}
  Texto imagen:    {0: 524, 1: 29, 2: 3}

HUMANO
  Texto analizado: {0: 2047, 1: 129, 2: 11}
  Texto imagen:    {0: 498, 1: 53, 2: 5}


**Coincidencias modelo vs humano**

In [ ]:
import json
from collections import Counter

# Indexar por ID
modelo_dict = {item["id"]: item for item in datos_modelo}
humano_dict = {item["id"]: item for item in datos_humano}

# Comparar agresividad
def comparar_agresividad_por_id(modelo_dict, humano_dict):
    coinciden = {"texto_analizado": Counter(), "texto_imagen": Counter()}
    no_coinciden = {"texto_analizado": Counter(), "texto_imagen": Counter()}

    ids_comunes = set(modelo_dict.keys()) & set(humano_dict.keys())

    for id_ in set(modelo_dict.keys()) & set(humano_dict.keys()):
        m = modelo_dict[id_]
        h = humano_dict[id_]

        for campo in ["texto_analizado", "texto_imagen"]:
            m_res = m.get("analisis_modelos", {}).get(campo, {}).get("resultados", {})
            h_res = h.get("analisis_modelos", {}).get(campo, {}).get("resultados", {})

            label_modelo = m_res.get("agresividad", {}).get("label")
            label_humano = h_res.get("agresividad")

            if label_modelo is not None and label_humano is not None:
                if label_modelo == label_humano:
                    coinciden[campo][label_humano] += 1
                else:
                    no_coinciden[campo][label_humano] += 1

    return coinciden, no_coinciden

# Ejecutar
coinciden, no_coinciden = comparar_agresividad_por_id(modelo_dict, humano_dict)

# Mostrar resultados
print("Coincidencias:")
for campo, conteo in coinciden.items():
    print(f"  {campo}: {dict(conteo)}")

print("\nNo coincidencias:")
for campo, conteo in no_coinciden.items():
    print(f"  {campo}: {dict(conteo)}")


Coincidencias:
  texto_analizado: {0: 2078, 1: 2}
  texto_imagen: {0: 487}

No coincidencias:
  texto_analizado: {0: 58, 2: 11, 1: 38}
  texto_imagen: {0: 32, 1: 32, 2: 5}


## **PÚBLICO OBJETIVO**

In [ ]:
import json
from collections import Counter

# === Función de conteo con lógica de neutro para el modelo ===
def contar_edad_genero(data, fuente):
    conteo = {}

    for item in data:
        for campo in ["texto_analizado", "texto_imagen"]:
            entrada = item.get("analisis_modelos", {}).get(campo, {})
            resultados = entrada.get("resultados", {})

            edad_key = f"edad_{campo}"
            genero_key = f"genero_{campo}"

            # Asegurar claves
            if edad_key not in conteo:
                conteo[edad_key] = Counter()
            if genero_key not in conteo:
                conteo[genero_key] = Counter()

            if fuente == "modelo":
                pub = resultados.get("publico_objetivo", {})

                # --- Edad con lógica de neutro ---
                probs_edad = pub.get("probs_edad", [])
                if len(probs_edad) == 3:
                    max_val = max(probs_edad)
                    diffs = [abs(max_val - v) for v in probs_edad]
                    if all(d < 0.25 for v, d in zip(probs_edad, diffs) if v != max_val):
                        conteo[edad_key]["neutro"] += 1
                    else:
                        idx = probs_edad.index(max_val)
                        conteo[edad_key][["18-29", "30-39", "40-49"][idx]] += 1

                # --- Género con lógica de neutro ---
                probs_genero = pub.get("probs_genero", [])
                if len(probs_genero) == 2:
                    dif = abs(probs_genero[0] - probs_genero[1])
                    if dif < 0.25:
                        conteo[genero_key]["neutro"] += 1
                    else:
                        idx = probs_genero.index(max(probs_genero))
                        conteo[genero_key][["male", "female"][idx]] += 1

            else:
                # Humano: edad y genero están directamente en resultados
                edad = resultados.get("edad")
                genero = resultados.get("genero")
                if edad: conteo[edad_key][edad] += 1
                if genero: conteo[genero_key][genero] += 1

    return conteo

# === Ejecutar ===
modelo_resultados = contar_edad_genero(datos_modelo, "modelo")
humano_resultados = contar_edad_genero(datos_humano, "humano")

# === Mostrar resultados ===
print("\n📊 RESULTADOS - ETIQUETAS DE EDAD Y GÉNERO\n")

for fuente, resultados in [("MODELO", modelo_resultados), ("HUMANO", humano_resultados)]:
    print(f"🔹 {fuente}")
    for clave, conteo in resultados.items():
        print(f"  {clave}: {dict(conteo)}")
    print()



📊 RESULTADOS - ETIQUETAS DE EDAD Y GÉNERO

🔹 MODELO
  edad_texto_analizado: {'neutro': 559, '40-49': 1292, '18-29': 296, '30-39': 40}
  genero_texto_analizado: {'male': 1175, 'neutro': 679, 'female': 333}
  edad_texto_imagen: {'40-49': 327, '18-29': 115, 'neutro': 112, '30-39': 2}
  genero_texto_imagen: {'male': 227, 'female': 154, 'neutro': 175}

🔹 HUMANO
  edad_texto_analizado: {'neutro': 1705, '18-29': 159, '30-39': 304, '40-49': 19}
  genero_texto_analizado: {'neutro': 1986, 'female': 125, 'male': 76}
  edad_texto_imagen: {'neutro': 334, '30-39': 159, '18-29': 58, '40-49': 5}
  genero_texto_imagen: {'neutro': 497, 'male': 24, 'female': 35}



**Coincidencias modelo vs humano**

In [ ]:
import json
from collections import Counter

# Función para decidir "neutro" si las diferencias entre probabilidades son pequeñas
def clasificar_con_neutro(probs, etiquetas, umbral=0.25):
    if not probs:
        return None
    max_val = max(probs)
    diffs = [abs(max_val - p) for p in probs]
    if all(d < umbral for p, d in zip(probs, diffs) if p != max_val):
        return "neutro"
    else:
        return etiquetas[probs.index(max_val)]

# Comparación por ID, con detección de neutro para el modelo
def comparar_publico_objetivo_por_id(modelo_dict, humano_dict):
    campos = ["texto_analizado", "texto_imagen"]
    resultados = {
        "edad": {"coinciden": {campo: Counter() for campo in campos},
                 "no_coinciden": {campo: Counter() for campo in campos}},
        "genero": {"coinciden": {campo: Counter() for campo in campos},
                   "no_coinciden": {campo: Counter() for campo in campos}},
    }

    ids_comunes = set(modelo_dict.keys()) & set(humano_dict.keys())

    for id_ in ids_comunes:
        m = modelo_dict[id_]
        h = humano_dict[id_]

        for campo in campos:
            m_res = m.get("analisis_modelos", {}).get(campo, {}).get("resultados", {}).get("publico_objetivo", {})
            h_res = h.get("analisis_modelos", {}).get(campo, {}).get("resultados", {})

            # === EDAD ===
            edad_m = clasificar_con_neutro(m_res.get("probs_edad"), ["18-29", "30-39", "40-49"])
            edad_h = h_res.get("edad")

            if edad_m and edad_h:
                if edad_m == edad_h:
                    resultados["edad"]["coinciden"][campo][edad_h] += 1
                else:
                    resultados["edad"]["no_coinciden"][campo][edad_h] += 1

            # === GENERO ===
            genero_m = clasificar_con_neutro(m_res.get("probs_genero"), ["male", "female"])
            genero_h = h_res.get("genero")

            if genero_m and genero_h:
                if genero_m == genero_h:
                    resultados["genero"]["coinciden"][campo][genero_h] += 1
                else:
                    resultados["genero"]["no_coinciden"][campo][genero_h] += 1

    return resultados


In [ ]:
# Convertir a diccionario por ID
modelo_dict = {item["id"]: item for item in datos_modelo}
humano_dict = {item["id"]: item for item in datos_humano}

# Ejecutar comparación
res = comparar_publico_objetivo_por_id(modelo_dict, humano_dict)

# Mostrar resultados
for tipo in ["edad", "genero"]:
    print(f"\nComparación de {tipo.upper()}")
    for estado in ["coinciden", "no_coinciden"]:
        print(f"{estado.capitalize()}:")
        for campo, conteo in res[tipo][estado].items():
            print(f"    {campo}: {dict(conteo)}")



Comparación de EDAD
Coinciden:
    texto_analizado: {'neutro': 445, '18-29': 4, '40-49': 14, '30-39': 4}
    texto_imagen: {'neutro': 90, '40-49': 4}
No_coinciden:
    texto_analizado: {'neutro': 1260, '30-39': 300, '18-29': 155, '40-49': 5}
    texto_imagen: {'neutro': 244, '30-39': 159, '18-29': 58, '40-49': 1}

Comparación de GENERO
Coinciden:
    texto_analizado: {'neutro': 620, 'male': 70, 'female': 37}
    texto_imagen: {'neutro': 158, 'female': 20, 'male': 12}
No_coinciden:
    texto_analizado: {'neutro': 1366, 'female': 88, 'male': 6}
    texto_imagen: {'neutro': 339, 'female': 15, 'male': 12}
